## Creates binarized Seurat objects, integrates conditions and annotates genes by nearby peaks

In [1]:
# Input info
#cellr_in = "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/DUPI_april08_2021_Croker/MTblacklist_mtasnucl/" 
#outdir = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/gff_hg38_1_2/mergedSamples/"

# Parameters
cores = 24

gff_id = "A2_black"

annotations_indir = "/mnt/md0/isshamie/Projects/Mito_Trace/data/processed/annotation_granges/"
config_f = "/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/chip_paper/CHIP_aggr_samples.yaml"
out_f = "/mnt/md0/isshamie/Projects/Mito_Trace/output/CHIP_aggr/data/annotation/gff_A2/mergedSamples/merged_peaks.bed"

In [2]:
library(repr)
options(repr.plot.width=12, repr.plot.height=12)

In [3]:
library(yaml)

samps <- read_yaml(config_f)$samples_csv
samps

Warning message in readLines(file):
“incomplete final line found on '/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/chip_paper/CHIP_aggr_samples.yaml'”


$CHIP_b1
[1] "/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/CHIP_dec172021/CHIP_b1_dec172021.csv"

$CHIP_b2
[1] "/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/CHIP_dec172021/InputOnly/CHIP_b2_dec172021.csv"

$CHIP_a1
[1] "/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/CHIP_Input_nameFix_april08_2021.csv"

$cd34norm
[1] "/data/Mito_Trace/parameters/pipeline/cosmo_server_v02/cd34norm.csv"

In [4]:
annotations <- readRDS(file.path(annotations_indir, paste0(gff_id, ".annotationGranges.rds")))

In [5]:
annotations

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



GRanges object with 2765969 ranges and 25 metadata columns:
              seqnames        ranges strand |   source       type     score
                 <Rle>     <IRanges>  <Rle> | <factor>   <factor> <numeric>
        [1]       chr1   29554-31109      + |   HAVANA gene              NA
        [2]       chr1   29554-31097      + |   HAVANA transcript        NA
        [3]       chr1   29554-30039      + |   HAVANA exon              NA
        [4]       chr1   30564-30667      + |   HAVANA exon              NA
        [5]       chr1   30976-31097      + |   HAVANA exon              NA
        ...        ...           ...    ... .      ...        ...       ...
  [2765965] KI270734.1 138483-138667      - |  ENSEMBL CDS               NA
  [2765966] KI270734.1 138480-138482      - |  ENSEMBL stop_codon        NA
  [2765967] KI270734.1 161689-161852      - |  ENSEMBL UTR               NA
  [2765968] KI270734.1 161587-161626      - |  ENSEMBL UTR               NA
  [2765969] KI270734.1 13808

In [6]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
#library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)

plan("multiprocess", workers = cores)
options(future.globals.maxSize = 8000 * 1024^2)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
#plan("multiprocess", workers = workers)

Attaching SeuratObject


Attaching package: ‘data.table’


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked from ‘package:S4Vectors’:

    first, second



Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, set

## Merge all peaks

In [7]:
read.peaks <- function(curr.samp.f){
    print('curr.samp')
    print(curr.samp.f)
    print(stringr::str_replace(curr.samp.f, "barcodes.tsv", "peaks.bed"))
    peaks <- read.table(
      file = stringr::str_replace(curr.samp.f, "barcodes.tsv", "peaks.bed"),
      col.names = c("chr", "start", "end")
    )
    # convert to genomic ranges
    gr <- makeGRangesFromDataFrame(peaks)
    return(gr)
}


run.experiments <- function(sample_csv){
    curr.csv <- readr::read_csv(sample_csv)
    print(curr.csv)
    curr.grs <- sapply(curr.csv$barcode_f, read.peaks)
    return(curr.grs)
}

gr.full <- sapply(samps, run.experiments)


Rows: 3 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): sample_name, cellr_ID, fastq_ID, barcode_f, bam_f

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 3 × 5
  sample_name cellr_ID fastq_ID             barcode_f                      bam_f
  <chr>       <chr>    <chr>                <chr>                          <chr>
1 Control     Control  BC_10xATAC_rxn1b_neg /home/isaac/lewis/mito_lineag… /hom…
2 Flt3l       Flt3l    BC_10xATAC_rxn1b_pos /home/isaac/lewis/mito_lineag… /hom…
3 Input       Input    BC_10xATAC_rxn1      /home/isaac/lewis/mito_lineag… /hom…
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b1/MTBlacklist_A2/Control/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b1/MTBlacklist_A2/Control/outs/filtered_peak_bc_matrix/peaks.bed"
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b1/MTBlacklist_A2/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b1/MTBlacklist_A2/Flt3l/outs/filtered_peak_bc_matrix/pe

Rows: 1 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): sample_name, cellr_ID, fastq_ID, barcode_f, bam_f

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 1 × 5
  sample_name cellr_ID fastq_ID        barcode_f                           bam_f
  <chr>       <chr>    <chr>           <chr>                               <chr>
1 Input       Input    BC_10xATAC_rxn2 /home/isaac/lewis/mito_lineage/out… /hom…
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b2/MTBlacklist_A2/Input/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_dec172021_b2/MTBlacklist_A2/Input/outs/filtered_peak_bc_matrix/peaks.bed"


Rows: 2 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): sample_name, cellr_ID, fastq_ID, barcode_f, bam_f

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 2 × 5
  sample_name cellr_ID fastq_ID      barcode_f                             bam_f
  <chr>       <chr>    <chr>         <chr>                                 <chr>
1 Input       Flt3l    BC_10xATAC_Fp /home/isaac/lewis/mito_lineage/outpu… /hom…
2 Flt3l       Control  BC_10xATAC_Fn /home/isaac/lewis/mito_lineage/outpu… /hom…
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/filtered_peak_bc_matrix/peaks.bed"
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_april08_2021_Croker/MTblacklist/Control/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/CHIP_april08_2021_Croker/MTblacklist/Control/outs/filtered_peak_bc_matrix/peaks.bed"


Rows: 2 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): sample_name, cellr_ID, fastq_ID, barcode_f, bam_f

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 2 × 5
  sample_name cellr_ID fastq_ID      barcode_f                             bam_f
  <chr>       <chr>    <chr>         <chr>                                 <chr>
1 Control     P2       BC_10xATAC_P2 /home/isaac/lewis/mito_lineage/outpu… /hom…
2 Flt3l       J2       BC_10xATAC_J2 /home/isaac/lewis/mito_lineage/outpu… /hom…
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/jan21_2021/MTblacklist/P2/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/jan21_2021/MTblacklist/P2/outs/filtered_peak_bc_matrix/peaks.bed"
[1] "curr.samp"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/jan21_2021/MTblacklist/J2/outs/filtered_peak_bc_matrix/barcodes.tsv"
[1] "/home/isaac/lewis/mito_lineage/output/mtscATAC/data/jan21_2021/MTblacklist/J2/outs/filtered_peak_bc_matrix/peaks.bed"


In [8]:
gr.full <- unlist(gr.full, recursive=FALSE, use.names=FALSE)


In [9]:
gr.full.c <- gr.full[[1]]
if (length(gr.full)>1){
    for (i in 2:length(gr.full)){
      gr.full.c <- c(gr.full.c, gr.full[[i]])
    }
}
combined.peaks <- reduce(x = c(gr.full.c))

# Filter out bad peaks based on length
peakwidths <- width(combined.peaks)
combined.peaks <- combined.peaks[peakwidths  < 10000 & peakwidths > 20]
head(combined.peaks)

GRanges object with 6 ranges and 0 metadata columns:
      seqnames        ranges strand
         <Rle>     <IRanges>  <Rle>
  [1]     chr1    9866-10638      *
  [2]     chr1   16106-16366      *
  [3]     chr1   96454-96767      *
  [4]     chr1 180685-181403      *
  [5]     chr1 184126-184527      *
  [6]     chr1 191059-192097      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths

In [10]:
length(combined.peaks)

[1] 230725

## Are any peaks in the chrM

In [11]:
length(combined.peaks[seqnames(combined.peaks) != "chrM"])


[1] 230725

In [12]:
#combined.peaks[seqnames(combined.peaks) == "chrM"]

combined.peaks[seqnames(combined.peaks) == "chr1"]


GRanges object with 21709 ranges and 0 metadata columns:
          seqnames              ranges strand
             <Rle>           <IRanges>  <Rle>
      [1]     chr1          9866-10638      *
      [2]     chr1         16106-16366      *
      [3]     chr1         96454-96767      *
      [4]     chr1       180685-181403      *
      [5]     chr1       184126-184527      *
      ...      ...                 ...    ...
  [21705]     chr1 248924437-248928616      *
  [21706]     chr1 248930023-248930301      *
  [21707]     chr1 248942373-248942525      *
  [21708]     chr1 248944205-248944462      *
  [21709]     chr1 248944909-248946333      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths

## Remove any peaks in the chrM

In [13]:
combined.peaks = combined.peaks[seqnames(combined.peaks) != "chrM"]
length(combined.peaks)

[1] 230725

## Save merged peaks as bed

In [14]:
gr <- unlist(as(gr.full, "GRangesList"))

In [15]:
df <- data.frame(seqnames=seqnames(gr),
  starts=start(gr)-1,
  ends=end(gr),
  names=tibble::rowid_to_column(as.data.frame(gr), "names")$names,
  scores=c(rep(".", length(gr))),
  strands=strand(gr))

df = dplyr::mutate_if(df, is.numeric, as.integer)
df

seqnames,starts,ends,names,scores,strands
<fct>,<int>,<int>,<int>,<chr>,<fct>
chr1,9866,10609,1,.,*
chr1,180846,180976,2,.,*
chr1,184393,184458,3,.,*
chr1,191211,191970,4,.,*
chr1,267786,268257,5,.,*
chr1,271048,271558,6,.,*
chr1,586018,586368,7,.,*
chr1,605251,605792,8,.,*
chr1,778134,779417,9,.,*


In [16]:
write.table(df, file=out_f, quote=F, sep="\t", row.names=F, col.names=F)

Warning message in file(file, ifelse(append, "a", "w")):
“cannot open file '/mnt/md0/isshamie/Projects/Mito_Trace/output/CHIP_aggr/data/annotation/gff_A2/mergedSamples/merged_peaks.bed': No such file or directory”


ERROR: Error in file(file, ifelse(append, "a", "w")): cannot open the connection


In [ ]:
sessionInfo()

## Create expSignac

In [ ]:
# read.peaks <- function(curr.samp.f){
#     print('curr.samp')
#     print(curr.samp.f)
#     print(stringr::str_replace(curr.samp.f, "barcodes.tsv", "peaks.bed"))
#     peaks <- read.table(
#       file = stringr::str_replace(curr.samp.f, "barcodes.tsv", "peaks.bed"),
#       col.names = c("chr", "start", "end")
#     )
#     # convert to genomic ranges
#     gr <- makeGRangesFromDataFrame(peaks)
#     return(gr)
# }


# run.experiments <- function(sample_csv){
#     curr.csv <- readr::read_csv(sample_csv)
#     print(curr.csv)
#     curr.grs <- sapply(curr.csv$barcode_f, read.peaks)
#     return(curr.grs)
# }

# gr.full <- sapply(samps, run.experiments)



In [ ]:
expr.load <- function(sample_names, samples, cellr_in) {
    allSE = c() 

    samples_df <- cbind(sample_names, samples)
    print('samples_df')
    print(samples_df)
    for (row in 1:nrow(samples_df)){
        exp <- (samples_df[[row, "samples"]])
        name <- (samples_df[[row, "sample_names"]]) 

    #for (exp in samples) {
        print('exp')
        print(exp)
        print('name')
        print(name)
        barcode_path <- file.path(cellr_in, exp, "outs", "filtered_peak_bc_matrix", "barcodes.tsv")    
        barcodes <- readr::read_tsv(barcode_path, col_names = F) # %>% tidyr::unite(barcode)
        barcodes <- as.data.frame(barcodes) %>%  tibble::column_to_rownames(var="X1") %>% tibble::add_column(proj=name)
        frag_file <- file.path(cellr_in, exp, "outs", "fragments.tsv.gz")

        cells.meta.f <- file.path(cellr_in, exp, "outs", "singlecell.csv") 
        cells.meta <- as.data.frame(readr::read_csv(cells.meta.f)) %>% tibble::column_to_rownames(var="barcode") %>% tibble::add_column(proj=name)
        cells.meta <- cells.meta[rownames(cells.meta) %in% rownames(barcodes), ]

        # quantify multiome peaks in the scATAC-seq dataset


        print("Creating fragments object")
        frags.curr <- CreateFragmentObject(path = frag_file, cells= rownames(barcodes))
        #print(frags.curr)
        print("Quantifying peaks")
        ## Quantify peaks
        curr.counts <- FeatureMatrix(
          fragments = frags.curr,
          features = combined.peaks,
          cells = rownames(barcodes),
          process_n = cores
        )

        print("Creating chromatin assay")
        ## Create the objects and use simple filters
        curr_assay <- CreateChromatinAssay(curr.counts, fragments = frags.curr, min.cells = 10, min.features = 200)
        curr <- CreateSeuratObject(curr_assay, assay = "ATAC", project=name, meta.data=cells.meta)
        print('curr_assay')
        print(head(curr_assay))
        print('curr')
        print(head(curr[[]]))
        allSE = c(allSE, curr)
        #return(curr)
    }

    return(allSE)
}
    #allSE <- sapply(samples, create_frag, cellr_in=cellr_in)